# DeepAR

Finally, you consider a more complex alternative to the dense neural network architecture.

[DeepAR](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html) is a supervised learning forecasting algorithm using recurrent neural networks (RNN); developed by Salinas, Flunkert, Gasthaus from Amazon research, it can be used in Amazon SageMaker.

The main feature which makes the algorithm stand out is the ability to learn a single global model using many related time series, and to apply the model on potentially unseen new series. Think about an energy provider training DeepAR on individual household electrical consumption time series, and then applying inference to predict future consumption of new customers (that have no history to feed the model).

The way the algorithm is able to correlate different time series at different times is by accepting/creating categorical features that label either single time indices or whole series.
In the electrical consumption case, for example, day of the week, week of the year or being a holiday are labels of the first type (pertaining a single time tick, across all time series), while domain knowledge labels such as number of people in the household or number of rooms of the house are attached of course to whole individual series.

In practice, the model takes in a two-dimensional matrix of real-valued data, where rows represent time indices and columns different series, and on top of this it assigns a set of labels to each element of the matrix depending on its row and column.

For technical details about DeepAR we refer to the introductory paper [DeepAR: Probabilistic Forecasting with Autoregressive Recurrent Networks](https://arxiv.org/abs/1704.04110).

Note that we won't use multiple time series nor the extra categorical features for our toy example.

# Setup

In [ ]:
# To read data from S3
! pip install pandas s3fs --upgrade

Please, restart the kernel if this is the first time you run this notebook.

This is necessary to ensure that we can actually import the libraries we've just installed in the previous cells.

In [ ]:
import os
import json
import datetime
from dateutil.parser import parse

import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
from sagemaker.serializers import IdentitySerializer

In [ ]:
# Configuring the default size for matplotlib plots
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 6)

In [ ]:
# 'ml.m5.xlarge' is included in the AWS Free Tier
INSTANCE_TYPE = 'ml.m5.xlarge'

# Raw data gathering

You start, as before, with the final dataset that the processing pipeline deposits on S3.
In order to retrieve it, some authentication and path-related variables need to be declared.

In [ ]:
VALIDATION_END = '2019-12-31 23:59'
TRAIN_END = '2018-12-31 23:59'
NOW = '2020-05-31'

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session()
sagemaker_client = boto_session.client("sagemaker")
sagemaker_bucket = sagemaker_session.default_bucket()
main_prefix = "amld22-workshop-sagemaker"

data_bucket = f"s3://{sagemaker_bucket}/{main_prefix}/data"
modelling_data_bucket = f'{data_bucket}/modelling/deepar'

raw_data_s3_path = "s3://public-workshop/normalized_data/processed/2006_2022_data.parquet"

raw_df = pd.read_parquet(raw_data_s3_path)
resampled_df = raw_df.resample('D').sum()

In [ ]:
load_series = resampled_df[:NOW].Load
load_series.head()

Since you want to try hyperparameter tuning, you need to split the dataset into train, test and validation. As always, you place yourself at the beginning of 2020; The whole of 2019 will be the validation set.

After tuning, you'll want to re-train the best model with all available data, including the validation set; for this reason let's define `train_ext` as the union of training and validation.

<div class="alert alert-block alert-warning">
<b>Simplification.</b> 

This is true for any model, including the Fourier regression and dense neural network. In real life, you always want to use all the data to train the production model (unless old data are not relevant anymore).
</div>


In [ ]:
load_train = load_series[:TRAIN_END]
load_validation = load_series[TRAIN_END:VALIDATION_END]
load_train_ext = load_series[:VALIDATION_END]
load_test = load_series[VALIDATION_END:]

## Upload on S3

DeepAR needs all its time series to be parsed and stored as json files on S3. Each file contains information about the beginning and the actual values of the series; their periodicity will be provided as a model parameter.

These files are first written locally, then copied to S3 before deleting the local version.

In [ ]:
training_data =[
    {
        "start": str(load_train.index[0]),
        "target": load_train.tolist(),
    }
]
validation_data =[
    {
        "start": str(load_validation.index[0]),
        "target": load_validation.tolist(),
    }
]
training_ext_data =[
    {
        "start": str(load_train_ext.index[0]),
        "target": load_train_ext.tolist(),
    }
]
test_data =[
    {
        "start": str(load_test.index[0]),
        "target": load_test.tolist(),
    }
]

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [ ]:
write_dicts_to_file("train.json", training_data)
write_dicts_to_file("validation.json", validation_data)
write_dicts_to_file("train_ext.json", training_ext_data)
write_dicts_to_file("test.json", test_data)

In [ ]:
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = boto3.resource("s3").Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(f"File {s3_path} already exists.\nSet override to upload anyway.\n")
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print("Uploading file to {}".format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
copy_to_s3("train.json", f"{modelling_data_bucket}/train/train.json")
copy_to_s3("validation.json", f"{modelling_data_bucket}/validation/validation.json")
copy_to_s3("train_ext.json", f"{modelling_data_bucket}/train_ext/train_ext.json")
copy_to_s3("test.json", f"{modelling_data_bucket}/test/test.json")

# Modeling

You are now ready to define the DeepAR estimator object, denoted as `forecasting-deepar`.

In [ ]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", sagemaker_session.boto_region_name)

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=sagemaker_session,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type=INSTANCE_TYPE,
    max_run=60*30,
    max_wait=60*45,
    use_spot_instances=True # See https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html
)

## Hyperparameters

You look around the internet for a good set of hyperparameters to provide the model. You can refer to [this](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar-tuning.html) for the main hyperparameters and tuning metrics; [here](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar_hyperparameters.html) you can find an explanation of each parameter as well as their ranges and default values.

In [ ]:
default_hyperparams = {
    "time_freq": "1D",
    "epochs": "35", # 335
    "early_stopping_patience": "40",
    "mini_batch_size": "128",
    "learning_rate": "1E-3",
    "context_length": "7",
    "prediction_length": "1",
    "num_cells": "84",
    "num_layers":"2",
    "dropout_rate": "0.1",
    "embedding_dimension": "10",
    "likelihood": "student-T"
}

In [ ]:
estimator.set_hyperparameters(**default_hyperparams)

You are not satisfied, though. 

You find out that [Amazon SageMaker automatic model tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html) is a powerful tool for automatic hyperparameter tuning.

You just define a `HyperparameterTuner` object, provide it with the estimator and ranges for the parameters you want to tune, and specify how many trials to perform in total (`max_jobs`) and contemporarily (`max_parallel_jobs`).

Once you've run the job by calling its `fit` method, you can follow all trials in your SageMaker Dashboard.

![SageMaker dashboard](img/sagemaker_dashboard_annotated.png)

After reading about the billing model of the service, you decide not to be too crazy and just optimize over one hyperparameter.

<div class="alert alert-block alert-warning">
<b>Warning.</b> 

You can optimize over a wider grid - and in real life you would. We are not doing it here to keep it short and avoid charges.
</div>


In [ ]:
hyperparameter_ranges = {
    'epochs': IntegerParameter(200, 400),
    #"context_length": IntegerParameter(5, 14),
    #"mini_batch_size": IntegerParameter(64, 128),    
    #'learning_rate': ContinuousParameter(0.0001, 0.01),
    #"num_cells": IntegerParameter(60, 100),
    #"num_layers": IntegerParameter(1, 8),
    #"dropout_rate": ContinuousParameter(0, 0.2),
    #"embedding_dimension": IntegerParameter(1, 20)
}

In [ ]:
objective_metric_name = 'test:RMSE'
objective_metric_type = 'Minimize'
metric_definitions = [{
    'Name': 'test:RMSE',
    'Regex': 'test:RMSE=([0-9\\.]+)'
}]

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name=objective_metric_name,    
    hyperparameter_ranges=hyperparameter_ranges,
    objective_type=objective_metric_type,
    max_jobs=4,
    max_parallel_jobs=4
)
#tuner.fit({
#    "train": f"{modelling_data_bucket}/train", 
#    "test": f"{modelling_data_bucket}/validation"
#})
#job_name = tuner._current_job_name

After tuning has completed, you can inspect the effect of parameter values with respect to the chosen metric by help of the `HyperparameterTuningJobAnalytics` object and some code for nice plots.

In [ ]:
#tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(job_name)
#full_df = tuner_analytics.dataframe().sort_values('FinalObjectiveValue')
#full_df

In [ ]:
#plt.plot(full_df.sort_values("epochs")['epochs'], full_df.sort_values("epochs")["FinalObjectiveValue"])
#plt.title('Loss vs Epochs')
#plt.xlabel('epochs')
#plt.ylabel('loss')
#plt.show()

Finally, when you have found a satisfactory set of parameters, you can select it directly from the analytics output and plug it back into the estimator for training.

In [ ]:
#best_hyperparams = dict(full_df[full_df.columns.intersection(list(default_hyperparams.keys()))].iloc[0, :])

In [ ]:
#final_hyperparams = dict(default_hyperparams, **best_hyperparams)
#for k, v in final_hyperparams.items():  # for some reason tuning turns int params into float
#    if not isinstance(v, str):
#        if v == int(v):
#            final_hyperparams[k] = str(int(v))
#        else:
#            final_hyperparams[k] = str(v)
#final_hyperparams

In [ ]:
#estimator.set_hyperparameters(**final_hyperparams)

## Training

As mentioned before, you're going to want to use all available data for training, and this means using the `train_ext` dataset defined at the beginning.

In [ ]:
estimator.fit(
    inputs={
        "train": f"{modelling_data_bucket}/train_ext", 
        "test": f"{modelling_data_bucket}/validation"
    },
    wait=True
)

## Deployment

As you already did for previous models, you deploy your DeepAR instance on a managed endpoint as a `DeepARPredictor`.

In [ ]:
class DeepARPredictor(sagemaker.predictor.Predictor):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            serializer=IdentitySerializer(content_type="application/json"),
            **kwargs,
        )

    def predict(
        self,
        ts,
        cat=None,
        dynamic_feat=None,
        num_samples=100,
        return_samples=False,
        quantiles=["0.1", "0.5", "0.9"],
    ):
        """Requests the prediction of for the time series listed in `ts`, each with the (optional)
        corresponding category listed in `cat`.

        ts -- `pandas.Series` object, the time series to predict
        cat -- integer, the group associated to the time series (default: None)
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        return_samples -- boolean indicating whether to include samples in the response (default: False)
        quantiles -- list of strings specifying the quantiles to compute (default: ["0.1", "0.5", "0.9"])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_time = ts.index[-1] + ts.index.freq
        quantiles = [str(q) for q in quantiles]
        req = self.__encode_request(ts, cat, dynamic_feat, num_samples, return_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, ts.index.freq, prediction_time, return_samples)

    def __encode_request(self, ts, cat, dynamic_feat, num_samples, return_samples, quantiles):
        instance = series_to_dict(
            ts, cat if cat is not None else None, dynamic_feat if dynamic_feat else None
        )

        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "samples"] if return_samples else ["quantiles"],
            "quantiles": quantiles,
        }

        http_request_data = {"instances": [instance], "configuration": configuration}

        return json.dumps(http_request_data).encode("utf-8")

    def __decode_response(self, response, freq, prediction_time, return_samples):
        # we only sent one time series so we only receive one in return
        # however, if possible one will pass multiple time series as predictions will then be faster
        predictions = json.loads(response.decode("utf-8"))["predictions"][0]
        prediction_length = len(next(iter(predictions["quantiles"].values())))
        prediction_index = pd.date_range(
            start=prediction_time, freq=freq, periods=prediction_length
        )
        if return_samples:
            dict_of_samples = {"sample_" + str(i): s for i, s in enumerate(predictions["samples"])}
        else:
            dict_of_samples = {}
        return pd.DataFrame(
            data={**predictions["quantiles"], **dict_of_samples}, index=prediction_index
        )

    def set_frequency(self, freq):
        self.freq = freq


def encode_target(ts):
    return [x if np.isfinite(x) else "NaN" for x in ts]


def series_to_dict(ts, cat=None, dynamic_feat=None):
    """Given a pandas.Series object, returns a dictionary encoding the time series.

    ts -- a pands.Series object with the target time series
    cat -- an integer indicating the time series category

    Return value: a dictionary
    """
    obj = {"start": str(ts.index[0]), "target": encode_target(ts)}
    if cat is not None:
        obj["cat"] = cat
    if dynamic_feat is not None:
        obj["dynamic_feat"] = dynamic_feat
    return obj

In [ ]:
predictor = estimator.deploy(
    initial_instance_count=1, 
    instance_type=INSTANCE_TYPE, 
    predictor_cls=DeepARPredictor
)

## Prediction

Rolling, 1 day ahead prediction inferences on the test set; each time you can pass the whole time series up to the day before the target, since DeepAR uses all information it can, even without re-training every day.

You look at the MAPE performance on COVID period only, since that's the short-term horizon you are interested into.

In [ ]:
rolling_preds = [predictor.predict(ts=load_series[:i], quantiles=[0.5]) for i in load_series[load_train_ext.index[-1]:].index]

In [ ]:
prediction_series = pd.concat([p.loc[:, '0.5'] for p in rolling_preds])

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / y_pred)

prediction_df = pd.DataFrame({"actual": load_test, "predicted": prediction_series}).dropna()
mape = mean_absolute_percentage_error(prediction_df["actual"], prediction_df["predicted"])

plt.title(f"DeepAR | Covid MAPE: {mape:.2%}")
plt.plot(prediction_df["actual"], label='actual')
plt.plot(prediction_df["predicted"], label='predicted')
plt.legend()
plt.grid(0.4)
plt.show()

In [ ]:
rolling_mape_df = pd.DataFrame(
    {'rolling_mape': map(lambda w: mean_absolute_percentage_error(w["actual"], w["predicted"]),
                         prediction_df.rolling(7))},
    index=prediction_df.index)
plt.plot(rolling_mape_df.rolling_mape)
plt.title('Rolling MAPE (7-day window)')
plt.grid(0.4)
plt.show()

# Cleanup

If you’re ready to be done with this notebook, please run the cells below with `CLEANUP = True`.

This will remove the model and the hosted endpoint.

In [ ]:
CLEANUP = True

In [ ]:
if CLEANUP:
    predictor.delete_model()
    predictor.delete_endpoint()